# PyTorch Implementation

## Importing Libraries

In [46]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
import glob
from PIL import Image
import time

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report

import torch
from torch.utils.data import DataLoader, Subset, Dataset
from torchvision import transforms
from torchvision import models
from torch import nn
import torch.nn.utils.prune as prune



## Utility Functions


In [66]:
def get_model_path(indices):
    if indices < 0 or indices >= len(model_name):
        raise ValueError("Index out of range.")
    model_folder = Path(MODEL_DIR) / model_name[indices]
    pth_files = glob.glob(str(model_folder / model_suffix))
    
    print(f"Loaded Model from {pth_files[0]}")
    if not pth_files:
        raise FileNotFoundError(f"No '.pth' files found in {model_folder}.")
    return pth_files[0]  # Return the first matching `.pth` file

def get_data(data_dir, batch_size=8, test_size=0.2, num_workers=0, random_state=42):
    # Define transforms for the images
    data_transforms = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize the image to 224x224
        transforms.ToTensor(),  # Convert PIL image to PyTorch tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize using ImageNet stats
    ])
    
    # Initialize the dataset
    dataset = SignalDataset2D(data_dir=data_dir, transforms=data_transforms)
    
    # Create indices for train-test split
    indices = list(range(len(dataset)))
    train_indices, val_indices = train_test_split(
        indices, test_size=test_size, random_state=random_state, stratify=dataset.labels
    )
    
    # Create subsets for train and validation
    train_subset = Subset(dataset, train_indices)
    val_subset = Subset(dataset, val_indices)
    
    # Create DataLoader objects
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    
    return train_loader, val_loader
    
def get_data_kfold(data_dir, batch_size=8, num_workers=0, num_folds=5, fold_index=0, random_state=42):
    # Define transforms for the images
    data_transforms = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize the image to 224x224
        transforms.ToTensor(),  # Convert PIL image to PyTorch tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize using ImageNet stats
    ])
    
    # Initialize the dataset
    dataset = SignalDataset2D(data_dir=data_dir, transforms=data_transforms)
    
    # Prepare the labels for stratification
    labels = dataset.labels
    
    # Set up StratifiedKFold
    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=random_state)
    
    # Get the train/val indices for the chosen fold
    all_splits = list(skf.split(range(len(dataset)), labels))
    
    if fold_index < 0 or fold_index >= num_folds:
        raise ValueError(f"Invalid fold_index {fold_index}. Must be between 0 and {num_folds - 1}.")
    
    train_indices, val_indices = all_splits[fold_index]
    
    # Create subsets for train and validation
    train_subset = Subset(dataset, train_indices)
    val_subset = Subset(dataset, val_indices)
    
    # Create DataLoader objects
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    
    return train_loader, val_loader

def predict(model: torch.nn.Module, dataloader: DataLoader, device: torch.device, num_classes: int = 8):
    model.eval()
    
    all_predictions = []
    all_labels = []
    total_inference_time = 0  # To track total inference time
    
    with torch.no_grad():
        for data, target in dataloader:
            # Start the timer for inference
            start_time = time.time()
            
            # Move data and labels to the device
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            # Stop the timer after inference
            end_time = time.time()
            total_inference_time += (end_time - start_time)
            
            # Assuming the model's output is logits; apply softmax for class probabilities
            probabilities = torch.softmax(output, dim=1)
            predicted_labels = torch.argmax(probabilities, dim=1).cpu().numpy()
            
            # Collect predictions and true labels
            all_predictions.extend(predicted_labels)
            all_labels.extend(target.cpu().numpy())
    
    # Compute confusion matrix
    conf_matrix = confusion_matrix(all_labels, all_predictions, labels=list(range(num_classes)))
    
    # Calculate overall metrics
    accuracy = np.trace(conf_matrix) / np.sum(conf_matrix)
    report = classification_report(all_labels, all_predictions, labels=list(range(num_classes)), output_dict=True, zero_division=0)
    
    # Extract class-wise metrics
    precision = {f"class_{i}": report[str(i)]['precision'] for i in range(num_classes)}
    recall = {f"class_{i}": report[str(i)]['recall'] for i in range(num_classes)}
    f1_score = {f"class_{i}": report[str(i)]['f1-score'] for i in range(num_classes)}
    
    # Compute mean metrics across classes
    mean_precision = np.mean(list(precision.values()))
    mean_recall = np.mean(list(recall.values()))
    mean_f1 = np.mean(list(f1_score.values()))
    
    # Calculate inference speed
    num_batches = len(dataloader)
    avg_inference_time_per_batch = total_inference_time / num_batches if num_batches > 0 else 0
    total_samples = len(all_labels)
    avg_inference_time_per_sample = total_inference_time / total_samples if total_samples > 0 else 0
    
    return {
        "predictions": all_predictions,
        "accuracy": accuracy,
        "mean_precision": mean_precision,
        "mean_recall": mean_recall,
        "mean_f1": mean_f1,
        "conf_matrix": conf_matrix,
        "class_precision": precision,
        "class_recall": recall,
        "class_f1": f1_score,
        "labels": all_labels,
        "total_inference_time": total_inference_time,
        "avg_inference_time_per_batch": avg_inference_time_per_batch,
        "avg_inference_time_per_sample": avg_inference_time_per_sample
    }

## 1. Data Loading

In [26]:
class SignalDataset2D(Dataset):
    def __init__(self, data_dir, transforms=None):
        self.data_dir = Path(data_dir)
        self.transforms = transforms
        self.image_files = []
        self.labels = []
        
        # Verify the existence of class directories (0-7)
        for class_label in range(8):  # Check for classes 0-7
            class_dir = self.data_dir / str(class_label)
            if not class_dir.exists() or not class_dir.is_dir():
                raise FileNotFoundError(f"Class directory '{class_label}' does not exist in {data_dir}.")
            
            # Load all .png files for the class
            class_files = list(class_dir.glob("*.png"))
            self.image_files.extend(class_files)
            self.labels.extend([class_label] * len(class_files))
        
        if not self.image_files:
            raise ValueError(f"No images found in the dataset directory: {data_dir}")

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        image = Image.open(self.image_files[index]).convert("RGB")
        label = self.labels[index]
        
        # Apply transforms if provided, otherwise return original image
        if self.transforms:
            image = self.transforms(image)
        
        return image, label
    

DATA_DIR = Path(r'D:\AUNUUN JEFFRY MAHBUUBI\PROJECT AND RESEARCH\PROJECTS\35. Institute of Information\CODE\data')
train_loader, val_loader = get_data(DATA_DIR, batch_size=8, test_size=0.2, num_workers=0, random_state=42)

Data shape: torch.Size([8, 3, 224, 224]), Target shape: torch.Size([8])


## 2. Trained Model Loading

In [12]:
MODEL_DIR = Path(r'D:\AUNUUN JEFFRY MAHBUUBI\PROJECT AND RESEARCH\PROJECTS\35. Institute of Information\CODE\model')

model_name = ['PretrainAlexNet', 'PretrainAlexNetPruneL1', 'PretrainAlexNetPruneL2', 
              'PretrainMobileNetV2', 'PretrainMobileNetV2PruneL1', 'PretrainMobileNetV2PruneL2']

model_suffix = '*.pth'

model_path = get_model_path(0)
print(f"Model path: {model_path}")

Model path: D:\AUNUUN JEFFRY MAHBUUBI\PROJECT AND RESEARCH\PROJECTS\35. Institute of Information\CODE\model\PretrainAlexNet\last_model_40.pth


### PyTorch Model Definition

In [47]:
class PretrainedAlexNet2D(nn.Module):
    def __init__(self, num_classes=8, pretrained=True):  # Change num_classes to match your dataset
        super(PretrainedAlexNet2D, self).__init__()

        # Load the pretrained AlexNet model
        self.base_model = models.alexnet(pretrained=pretrained)

        # Modify the classifier's final layer for custom output classes
        if num_classes != 1000:  # ImageNet has 1000 classes
            self.base_model.classifier[6] = nn.Linear(4096, num_classes)

    def forward(self, x):
        return self.base_model(x)
class PretrainedAlexNet2DPrunedL1(nn.Module):
    def __init__(self, num_classes=1000, pretrained=True, pruning_amount=0.2):
        super(PretrainedAlexNet2DPrunedL1, self).__init__()

        # Load the pretrained AlexNet model
        self.base_model = models.alexnet(pretrained=pretrained)

        # Modify the classifier's final layer for custom output classes
        if num_classes != 1000:  # ImageNet has 1000 classes
            self.base_model.classifier[6] = nn.Linear(4096, num_classes)

        # Apply L1 pruning to convolutional layers
        for layer in self.base_model.features:
            if isinstance(layer, nn.Conv2d):
                prune.l1_unstructured(layer, name="weight", amount=pruning_amount)

    def forward(self, x):
        return self.base_model(x)
class PretrainedAlexNet2DPrunedL2(nn.Module):
    def __init__(self, num_classes=1000, pretrained=True, pruning_amount=0.2):
        super(PretrainedAlexNet2DPrunedL2, self).__init__()

        # Load the pretrained AlexNet model
        self.base_model = models.alexnet(pretrained=pretrained)

        # Modify the classifier's final layer for custom output classes
        if num_classes != 1000:  # ImageNet has 1000 classes
            self.base_model.classifier[6] = nn.Linear(4096, num_classes)

        # Apply structured pruning (L2-norm) to convolutional layers
        for layer in self.base_model.features:
            if isinstance(layer, nn.Conv2d):
                prune.ln_structured(layer, name="weight", amount=pruning_amount, n=2, dim=0)

    def forward(self, x):
        return self.base_model(x)
class PretrainedMobileNetV2(nn.Module):
    def __init__(self, num_classes=1000, pretrained=True):
        super(PretrainedMobileNetV2, self).__init__()

        # Load the pretrained MobileNetV2 model
        self.base_model = models.mobilenet_v2(pretrained=pretrained)

        # Modify the classifier's final layer for custom output classes
        if num_classes != 1000:  # ImageNet has 1000 classes
            self.base_model.classifier[1] = nn.Linear(self.base_model.classifier[1].in_features, num_classes)

    def forward(self, x):
        return self.base_model(x)
class PretrainedMobileNetV2WithL1Pruning(nn.Module):
    def __init__(self, num_classes=1000, pretrained=True, pruning_amount=0.2):
        super(PretrainedMobileNetV2WithL1Pruning, self).__init__()

        # Load the pretrained MobileNetV2 model
        self.base_model = models.mobilenet_v2(pretrained=pretrained)

        # Modify the classifier's final layer for custom output classes
        if num_classes != 1000:  # ImageNet has 1000 classes
            self.base_model.classifier[1] = nn.Linear(self.base_model.classifier[1].in_features, num_classes)

        # Apply L1 pruning to the layers
        for layer in self.base_model.features:
            if isinstance(layer, nn.Conv2d):
                prune.l1_unstructured(layer, name="weight", amount=pruning_amount)

    def forward(self, x):
        return self.base_model(x)
class PretrainedMobileNetV2WithL2Pruning(nn.Module):
    def __init__(self, num_classes=1000, pretrained=True, pruning_amount=0.2):
        super(PretrainedMobileNetV2WithL2Pruning, self).__init__()

        # Load the pretrained MobileNetV2 model
        self.base_model = models.mobilenet_v2(pretrained=pretrained)

        # Modify the classifier's final layer for custom output classes
        if num_classes != 1000:  # ImageNet has 1000 classes
            self.base_model.classifier[1] = nn.Linear(self.base_model.classifier[1].in_features, num_classes)

        # Apply L2 structured pruning to the layers
        for layer in self.base_model.features:
            if isinstance(layer, nn.Conv2d):
                prune.ln_structured(layer, name="weight", amount=pruning_amount, n=2, dim=0)

    def forward(self, x):
        return self.base_model(x)

In [78]:
# Define a mapping dictionary for models
model_mapping = {
    0: lambda: PretrainedAlexNet2D(num_classes=8),
    1: lambda: PretrainedAlexNet2DPrunedL1(num_classes=8, pruning_amount=0.2),
    2: lambda: PretrainedAlexNet2DPrunedL2(num_classes=8, pruning_amount=0.2),
    3: lambda: PretrainedMobileNetV2(num_classes=8),
    4: lambda: PretrainedMobileNetV2WithL1Pruning(num_classes=8, pruning_amount=0.2),
    5: lambda: PretrainedMobileNetV2WithL2Pruning(num_classes=8, pruning_amount=0.2),
}

# Define the indices for the model
indices = 3  # Change this to select the model dynamically

# Ensure indices are valid
if indices not in model_mapping:
    raise ValueError(f"Invalid index {indices}. Must be between 0 and {len(model_mapping) - 1}.")

# Initialize the model dynamically
model = model_mapping[indices]()  # Use the lambda function to instantiate the model

# Load the model weights
state_dict = torch.load(get_model_path(indices))  # Replace get_model_path(indices) with the correct path to the .pth file
model.load_state_dict(state_dict)

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Load data
# train_loader, val_loader = get_data(DATA_DIR, batch_size=8, test_size=0.2, num_workers=0, random_state=42)
train_loader, val_loader = get_data_kfold(DATA_DIR, batch_size=8, num_workers=0, num_folds=5, fold_index=4, random_state=42)

# Perform predictions
results = predict(model, val_loader, device, num_classes=8)

# Display overall results
print(f"Accuracy: {results['accuracy']:.2%}")  # Display as a percentage
print(f"Mean Precision: {results['mean_precision']:.2%}")
print(f"Mean Recall: {results['mean_recall']:.2%}")
print(f"Mean F1: {results['mean_f1']:.2%}")

# Display confusion matrix
print("\nConfusion Matrix:")
print(results['conf_matrix'])

# Display metrics for each class
print("\nClass-wise Metrics:")
for cls in sorted(results['class_precision'].keys()):
    precision = results['class_precision'][cls] * 100  # Convert to percentage
    recall = results['class_recall'][cls] * 100
    f1 = results['class_f1'][cls] * 100
    print(f"Class {cls}: Precision: {precision:.2f}%, Recall: {recall:.2f}%, F1-Score: {f1:.2f}%")

# Display inference time
print("\nInference Time:")
print(f"Total Inference Time: {results['total_inference_time']:.4f} seconds")
print(f"Average Inference Time per Batch: {results['avg_inference_time_per_batch']:.4f} seconds")
print(f"Average Inference Time per Sample: {results['avg_inference_time_per_sample']:.6f} seconds")



Loaded Model from D:\AUNUUN JEFFRY MAHBUUBI\PROJECT AND RESEARCH\PROJECTS\35. Institute of Information\CODE\model\PretrainMobileNetV2\last_model_40.pth
Accuracy: 66.25%
Mean Precision: 69.34%
Mean Recall: 66.25%
Mean F1: 66.24%

Confusion Matrix:
[[9 0 0 0 1 0 0 0]
 [2 6 1 0 0 1 0 0]
 [0 0 5 2 1 1 1 0]
 [0 0 0 5 2 2 0 1]
 [0 0 0 2 7 1 0 0]
 [1 0 0 1 0 7 1 0]
 [0 0 0 0 0 4 5 1]
 [0 1 0 0 0 0 0 9]]

Class-wise Metrics:
Class class_0: Precision: 75.00%, Recall: 90.00%, F1-Score: 81.82%
Class class_1: Precision: 85.71%, Recall: 60.00%, F1-Score: 70.59%
Class class_2: Precision: 83.33%, Recall: 50.00%, F1-Score: 62.50%
Class class_3: Precision: 50.00%, Recall: 50.00%, F1-Score: 50.00%
Class class_4: Precision: 63.64%, Recall: 70.00%, F1-Score: 66.67%
Class class_5: Precision: 43.75%, Recall: 70.00%, F1-Score: 53.85%
Class class_6: Precision: 71.43%, Recall: 50.00%, F1-Score: 58.82%
Class class_7: Precision: 81.82%, Recall: 90.00%, F1-Score: 85.71%

Inference Time:
Total Inference Time: 0.27